# Ideological Caucusses in congress
### Democrats
- Congressional Progressive Caucus (CPC) – liberal and progressive Democrats
- New Democrat Coalition (NDC) – moderate and centrist Democrats
- Blue Dog Coalition (BDC) – conservative Democrats
- Blue Collar Caucus – Pro-labor and Anti-Globalization Democrats

### Republicans
- Tuesday Group (TG) – moderate Republicans
- Republican Main Street Partnership (MSP) – another group of moderate Republicans
- Republican Study Committee (RSC) – conservative Republicans
- Liberty Caucus (LC) – libertarian Republicans
- House Freedom Caucus (HFC) – a conservative caucus affiliated with the Tea Party movement

## Scrape them Dem caucusses
What they all have in common is, that they are displayed as a link in each of the articles. 
>1. extract all links from the pages
>2. use Congress API to make a list with members of the current Congress
>3. take only the names from (1) that are found in (2)

In [244]:
import wikipedia as wp
import pandas as pd
import regex as re
from congress import Congress
from bs4 import BeautifulSoup

In [245]:
congress = Congress('qa1qfTzGinGGLcNCbinp2OTlI2SvzBe4KFgxYobm')

In [260]:
congress.members.filter('house', congress=115,)[0]['members'][0]

{'api_uri': 'https://api.propublica.org/congress/v1/members/A000374.json',
 'at_large': False,
 'contact_form': None,
 'crp_id': 'N00036633',
 'cspan_id': '76236',
 'date_of_birth': '1954-09-16',
 'district': '5',
 'dw_nominate': 0.497,
 'facebook_account': 'CongressmanRalphAbraham',
 'fax': '202-225-5639',
 'fec_candidate_id': 'H4LA05221',
 'first_name': 'Ralph',
 'gender': 'M',
 'geoid': '2205',
 'google_entity_id': '/m/012dwd7_',
 'govtrack_id': '412630',
 'icpsr_id': '21522',
 'id': 'A000374',
 'ideal_point': None,
 'in_office': True,
 'last_name': 'Abraham',
 'leadership_role': None,
 'middle_name': None,
 'missed_votes': 8,
 'missed_votes_pct': 0.9,
 'next_election': '2018',
 'ocd_id': 'ocd-division/country:us/state:la/cd:5',
 'office': '417 Cannon House Office Building',
 'party': 'R',
 'phone': '202-225-8490',
 'rss_url': 'https://abraham.house.gov/rss.xml',
 'seniority': '4',
 'short_title': 'Rep.',
 'state': 'LA',
 'suffix': None,
 'title': 'Representative',
 'total_present':

In [270]:
def get_names(congress_number):
    memberlist = congress.members.filter('house', congress=congress_number,)[0]['members']
    namelist = [] 
    name_part = ['first_name', 'middle_name', 'last_name']
    name_fragment = ""
    for m in memberlist:  
        #skip if not in office 
        if m['in_office'] == False: 
            continue
        else:
            for n in name_part:
                if m[n] == None: #catch nones
                    name_fragment += ""
                elif n != 'last_name': #add space, except after last_name
                    name_fragment += m[n]+" "
                else: #no space after last_name
                    name_fragment += m[n]
            namelist.append(name_fragment)
            name_fragment = "" #reset temp variable name_fragment
    return namelist

In [275]:
#all members of the 115 congress
members_of_115 = get_names(115)

In [291]:
#these are exactly the titles of the Wikipedia articles
wiki_dem_caucus = ['Congressional Progressive Caucus', 
                   'New Democrat Coalition', 
                   'List of members of the Blue Dog Coalition', 
                   'Blue Collar Caucus']

In [292]:
def caucus_members(caucus, namelist):
    #get all links from the website 'caucus'
    page = wp.page(caucus)
    soup = BeautifulSoup(page.html(), 'lxml')
    rawlinks = soup.find_all('a') #to retrieve all links
    linktexts = [li.text for li in rawlinks] #save link texts into list
    #get intersection of linktexts and namelist to have all caucus members extracted that serve currently
    setlinks = set(linktexts)
    setnames = set(namelist)
    current_serve = setnames.intersection(setlinks)
    return current_serve

In [295]:
dem_caucus_dict = {cauc: caucus_members(cauc, members_of_115) for cauc in wiki_dem_caucus}

In [298]:
dem_caucus_dict

{'Blue Collar Caucus': {'Alan Lowenthal',
  'Bonnie Watson Coleman',
  'Brendan Boyle',
  'Brian Higgins',
  'Carol Shea-Porter',
  'Chellie Pingree',
  'Cheri Bustos',
  'Dan Kildee',
  'Daniel Lipinski',
  'Debbie Dingell',
  'Derek Kilmer',
  'Dina Titus',
  'Eric Swalwell',
  'Jan Schakowsky',
  'Joe Courtney',
  'John Yarmuth',
  'Kathy Castor',
  'Marc Veasey',
  'Marcy Kaptur',
  'Mark Pocan',
  'Matt Cartwright',
  'Paul Tonko',
  'Rick Nolan',
  'Ruben Gallego',
  'Seth Moulton',
  'Tim Ryan',
  'Vicente Gonzalez'},
 'Congressional Progressive Caucus': {'Adam Smith',
  'Adriano Espaillat',
  'Alan Lowenthal',
  'Alma Adams',
  'André Carson',
  'Barbara Lee',
  'Bennie Thompson',
  'Bonnie Watson Coleman',
  'Brenda Lawrence',
  'Carol Shea-Porter',
  'Chellie Pingree',
  'Darren Soto',
  'Dave Loebsack',
  'David Cicilline',
  'Debbie Dingell',
  'Dwight Evans',
  'Eddie Bernice Johnson',
  'Eleanor Holmes Norton',
  'Frank Pallone',
  'Frederica Wilson',
  'Gwen Moore',
  'H